<a href="https://colab.research.google.com/github/nebulousman/labeling_reviews/blob/main/labelingexport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Select Random Samples of Reviews for Labeling**
Dariush Moghadam


This code was created for CrowdDoing and the effort toward manually-curated “gold” labels of reviews for training NLP models. The goal is an automated script that randomly samples the review dataset and writes to excel document which can be easily distributed to anyone for labeling. Excel is selected for two reasons: 1) most people have the software; and 2) the ability to restrict input values and maintain data integrity.


In [ ]:
#connect to google drive for dataset
# mount google drive for access to dataset previously added to the drive
from google.colab import drive
drive.mount('/content/drive')
#if you need to remount drive
#drive.mount('/content/drive', force_remount=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#imports
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Alignment, Border, Side
import time

In [ ]:
#define function that writes dataframe to excel spreadsheet
def createXLfile(sampledata):
    book = load_workbook('/content/drive/My Drive/Analysis/labeling_reviews/master_files/Master.xlsx')
    writer = pd.ExcelWriter('/content/drive/My Drive/Analysis/labeling_reviews/samples_to_label/Reviews'+time.strftime('%Y%m%d_%H%M%S')+'.xlsx', engine='openpyxl')
    writer.book = book
    writer.sheets = {ws.title: ws for ws in book.worksheets}
    sampledata.to_excel(writer, sheet_name='Review_Labels', startrow=0, index=False)
    for cell in writer.sheets['Review_Labels']['B']:
        cell.alignment = Alignment(wrap_text=True)
        cell.border = Border(bottom=Side(border_style='thin', color='000000'))
    writer.save()

In [ ]:
# read dataset into dataframe - change path if needed
# note CrowdDoingAmazonReviewsFORSampling.csv is 2 columns of CrowdDoingAmazonReviews.csv - the ReviewContent + the ID which is the index
# CrowdDoingAmazonReviewsFORSampling.csv is updated every time this notebook runs. Samples removed for labeling are removed from
# CrowdDoingAmazonReviewsFORSampling.csv so they're not picked again - i.e. the updated file is loaded with this code block.
# labeled reviews returned by volunteers can be mapped back to the original CrowdDoingAmazonReviews.csv using the ID index value
df = pd.read_csv('/content/drive/My Drive/Analysis/labeling_reviews/master_files/CrowdDoingAmazonReviewsFORSampling.csv')

In [ ]:
#explore imported file
#df.head()
#df.dtypes
len(df)

531056

In [ ]:
#set count of files and sample size of each file
#samplecount is number or count of files to output & samplesize is number of records or rows per file
samplecount=100
samplesize=20

In [ ]:
for i in range(samplecount):
    df_subset=df.sample(samplesize)
    createXLfile(df_subset)
    df=df.drop(df_subset.index)
    time.sleep(1) #added because it may run too fast for unique naming thereby overwriting - note samplecount value/60 will be approx minutes this will run

/usr/local/lib/python3.6/dist-packages/openpyxl/reader/worksheet.py:322: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [ ]:
#compare to prior one - confirm samples were removed
len(df)

529056

In [ ]:
#save remaining reviews (not sampled) for future sampling with this code
df.to_csv('/content/drive/My Drive/Analysis/labeling_reviews/master_files/CrowdDoingAmazonReviewsFORSampling.csv', index=False)